## Project non-payment probability

### Objective:
- The objective of this work is to train a classification model that can inform us of the probability of default occurring on a financial loan.

- O objetivo desse trabalho é treinar um modelo de classificação que possa nos informar qual a probabilidade de inadimplência ocorrer em um empréstimo financeiro.

### Data Origin:
- Dataset: https://www.kaggle.com/datasets/itssuru/loan-data

- publicly available data from LendingClub.com. Lending Club connects people who need money (borrowers) with people who have money (investors). Hopefully, as an investor you would want to invest in people who showed a profile of having a high probability of paying you back.

- Dados disponíveis publicamente do LendingClub.com. O Lending Club conecta pessoas que precisam de dinheiro (tomadores de empréstimo) com pessoas que têm dinheiro (investidores). Felizmente, como investidor, você gostaria de investir em pessoas que mostrassem um perfil de ter uma alta probabilidade de pagar você de volta.

- Aqui está o que as colunas representam:

    credit.policy: 1 se o cliente atender aos critérios de subscrição de crédito do LendingClub.com e 0 caso contrário.

    purpose: O propósito do empréstimo (assume os valores "credit_card", "debt_consolidation", "educational", "major_purchase", "small_business" e "all_other").

    int.rate: A taxa de juros do empréstimo, como uma proporção (uma taxa de 11% seria armazenada como 0,11). Os mutuários considerados pelo LendingClub.com como mais arriscados recebem taxas de juros mais altas.

    installment: As parcelas mensais devidas pelo mutuário se o empréstimo for financiado.

    log.annual.inc: O logaritmo natural da renda anual autodeclarada do mutuário.

    dti: A relação dívida/renda do mutuário (valor da dívida dividido pela renda anual).

    fico: A pontuação de crédito FICO do mutuário.

    days.with.cr.line: O número de dias em que o mutuário teve uma linha de crédito.

    revol.bal: Saldo rotativo do mutuário (valor não pago no final do ciclo de cobrança do cartão de crédito).

    revol.util: Taxa de utilização da linha rotativa do mutuário (o valor da linha de crédito usada em relação ao crédito total disponível).

    inq.last.6mths: Número de consultas do mutuário por credores nos últimos 6 meses.

    delinq.2yrs: Número de vezes que o mutuário atrasou um pagamento em mais de 30 dias nos últimos 2 anos.

    pub.rec: Número de registros públicos depreciativos do mutuário (declarações de falência, penhoras fiscais ou julgamentos).

    not.fully.paid: O empréstimo não foi totalmente pago.

- ## Análise exploratória dos dados
- ## Preparação dos dados
- ## Armazenamento dos dados tratados

In [1]:
# maximiza nro de linhas e colunas para exibição
# inibe mensagens de warning
import pandas as pd
pd.set_option('display.max_rows', None) # permite a máxima visualização das linhas em um display
pd.set_option('display.max_columns', None) # permite a máxima visualização das colunas em um display
import warnings
warnings.simplefilter('ignore') # inibe a exibição de avisos de warning

In [2]:
# métricas de verificação
from scipy.stats import skew # verifica se a base do sino da distribuição normal está entre -2 e 2 que é o limite viável
from scipy.stats import kurtosis # verifica se o cume do sino da distribuição normal está entre -2 e 2 (limite viável)

In [3]:
# preparação e visualização de dados
from sklearn.preprocessing import StandardScaler 
                    # faz a padronização dos dados para melhorar a normalidade (média=0 e variância=1)
                    # sensível a outliers

from sklearn.compose import ColumnTransformer # Aplica transformadores a colunas de um array ou pandas DataFrame

from fc import fc_retira_outliers as rto # função que retira outliers

from matplotlib import pyplot # plotagem gráfica
import seaborn as sns # plotagem gráfica baseada em matplotlib
import numpy as np # computaçaõ de matrizes

In [4]:
# carregando arquivo de dados de cotações financeiras
df_emprestimo = pd.read_csv('dataset/loan_data.csv', sep=',')

In [5]:
df_emprestimo.shape

(9578, 14)

In [6]:
df_emprestimo.head(3)

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0


In [7]:
# reconhecendo as variáveis disponíveis
df_emprestimo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   int64  
 1   purpose            9578 non-null   object 
 2   int.rate           9578 non-null   float64
 3   installment        9578 non-null   float64
 4   log.annual.inc     9578 non-null   float64
 5   dti                9578 non-null   float64
 6   fico               9578 non-null   int64  
 7   days.with.cr.line  9578 non-null   float64
 8   revol.bal          9578 non-null   int64  
 9   revol.util         9578 non-null   float64
 10  inq.last.6mths     9578 non-null   int64  
 11  delinq.2yrs        9578 non-null   int64  
 12  pub.rec            9578 non-null   int64  
 13  not.fully.paid     9578 non-null   int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


In [8]:
# renomeando as colunas para facilitar o trabalho
df_emprestimo.columns = ['atende_criterios', 'utilizacao', 'tx_juros', 'vr_parcelas', 'log_renda_anual', \
                         'idx_div_renda', 'pont_mutuario', 'dias_credito', 'saldo_rotativo', 'tx_util_credito', \
                         'consult_mutuario_cred', 'nro_atraso', 'nro_reg_deprec', 'devedor']

In [9]:
df_emprestimo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   atende_criterios       9578 non-null   int64  
 1   utilizacao             9578 non-null   object 
 2   tx_juros               9578 non-null   float64
 3   vr_parcelas            9578 non-null   float64
 4   log_renda_anual        9578 non-null   float64
 5   idx_div_renda          9578 non-null   float64
 6   pont_mutuario          9578 non-null   int64  
 7   dias_credito           9578 non-null   float64
 8   saldo_rotativo         9578 non-null   int64  
 9   tx_util_credito        9578 non-null   float64
 10  consult_mutuario_cred  9578 non-null   int64  
 11  nro_atraso             9578 non-null   int64  
 12  nro_reg_deprec         9578 non-null   int64  
 13  devedor                9578 non-null   int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB

In [10]:
df_emprestimo.head(10)

,atende_criterios,utilizacao,tx_juros,vr_parcelas,log_renda_anual,idx_div_renda,pont_mutuario,dias_credito,saldo_rotativo,tx_util_credito,consult_mutuario_cred,nro_atraso,nro_reg_deprec,devedor
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
5,1,credit_card,0.0788,125.13,11.904968,16.98,727,6120.041667,50807,51.0,0,0,0,0
6,1,debt_consolidation,0.1496,194.02,10.714418,4.00,667,3180.041667,3839,76.8,0,0,1,1
7,1,all_other,0.1114,131.22,11.002100,11.08,722,5116.000000,24220,68.6,0,0,0,1
8,1,home_improvement,0.1134,87.19,11.407565,17.25,682,3989.000000,69909,51.1,1,0,0,0
9,1,debt_consolidation,0.1221,84.12,10.203592,10.00,707,2730.041667,5630,23.0,1,0,0,0


In [11]:
print('utilizacao', df_emprestimo.atende_criterios.value_counts())
print('consult_mutuario_cred', df_emprestimo.consult_mutuario_cred.value_counts())
print('nro_atraso', df_emprestimo.nro_atraso.value_counts())
print('nro_reg_deprec', df_emprestimo.nro_reg_deprec.value_counts())
print('devedor', df_emprestimo.devedor.value_counts())

utilizacao atende_criterios
1    7710
0    1868
Name: count, dtype: int64
consult_mutuario_cred consult_mutuario_cred
0     3637
1     2462
2     1384
3      864
4      475
5      278
6      165
7      100
8       72
9       47
10      23
11      15
12      15
15       9
13       6
14       6
18       4
16       3
19       2
24       2
17       2
33       1
20       1
27       1
25       1
28       1
31       1
32       1
Name: count, dtype: int64
nro_atraso nro_atraso
0     8458
1      832
2      192
3       65
4       19
5        6
6        2
13       1
7        1
8        1
11       1
Name: count, dtype: int64
nro_reg_deprec nro_reg_deprec
0    9019
1     533
2      19
3       5
4       1
5       1
Name: count, dtype: int64
devedor devedor
0    8045
1    1533
Name: count, dtype: int64
